In [80]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [81]:
x = np.load('data/EEG_all_epochs.npy')
y = np.load('data/EEG_contaminated_with_EOG.npy')

In [82]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [83]:
model = Sequential()
model.add(Dense(512, input_dim=y_train.shape[1], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='linear'))

model.compile(loss = 'mean_squared_error', optimizer = 'Nadam')

model.fit(y_train, x_train, epochs = 300, batch_size = 64, verbose = 1)

Epoch 1/300
64/64 [==============================] - 2s 7ms/step - loss: 46076.9805
Epoch 2/300
64/64 [==============================] - 0s 7ms/step - loss: 25942.7832
Epoch 3/300
64/64 [==============================] - 0s 7ms/step - loss: 18416.3789
Epoch 4/300
64/64 [==============================] - 0s 7ms/step - loss: 14803.2500
Epoch 5/300
64/64 [==============================] - 0s 8ms/step - loss: 12786.5146
Epoch 6/300
64/64 [==============================] - 1s 9ms/step - loss: 11389.5977
Epoch 7/300
64/64 [==============================] - 1s 8ms/step - loss: 10159.9277
Epoch 8/300
64/64 [==============================] - 0s 7ms/step - loss: 9330.6055
Epoch 9/300
64/64 [==============================] - 0s 6ms/step - loss: 8673.9463
Epoch 10/300
64/64 [==============================] - 0s 6ms/step - loss: 8023.3359
Epoch 11/300
64/64 [==============================] - 0s 7ms/step - loss: 7713.5859
Epoch 12/300
64/64 [==============================] - 0s 7ms/step - loss: 7422

In [84]:
y_pred = model.predict(x_val)

# Calcular el error cuadrático medio entre las predicciones y los datos de validación limpios
mse = mean_squared_error(y_val, y_pred)
print("MSE:", mse)

# Calcular el coeficiente de correlación entre las predicciones y los datos de validación limpios
corr = np.corrcoef(y_val.reshape(-1), y_pred.reshape(-1))[0, 1]
print("Coeficiente de correlación:", corr)

15/15 [==============================] - 0s 7ms/step
MSE: 77317.25823064725
Coeficiente de correlación: 0.6170443321717616


Adam: 0.60
Adamax: 0.61
Nadam: 0.61